# Rasters and Tiles

Overview of basic types in GeoTrellis and their quirks. This workbook mostly tracks [Tiles](https://geotrellis.github.io/geotrellis-workshop/docs/tiles) from the workshop docs.



In [ ]:
import $ivy.`org.slf4j:slf4j-simple:1.7.30`
import $ivy.`org.locationtech.geotrellis::geotrellis-raster:3.5.2`

import geotrellis.raster._

- [`Tile`](https://github.com/locationtech/geotrellis/blob/master/raster/src/main/scala/geotrellis/raster/Tile.scala) is fundemental interface in GeoTrellis. 
- Most common type of `Tile` is [`ArrayTile`](https://github.com/locationtech/geotrellis/blob/master/raster/src/main/scala/geotrellis/raster/ArrayTile.scala) -- it is backed by Scala `Array` type.
- Any primitive type can back a `Tile` (`Int`, `Short`, `Double`, `Float`, `Byte`)
- While `Array`s are mutable, `Tile` interface does not allow mutation.
- Performance is primary concern behind the `Tile` interface design

In [ ]:
val rawData = 1 to 16 toArray
val tile16 = ArrayTile(rawData, cols = 4, rows = 4)
Text(tile16.asciiDraw())

## CellType

Tile also has to have a [`CellType`](https://github.com/locationtech/geotrellis/blob/master/raster/src/main/scala/geotrellis/raster/CellType.scala) in order to carry the `NODATA` value and keep track of unsigned types. Its string representation encodes the nodata type.

- [NoDatas per data type](https://github.com/locationtech/geotrellis/blob/1a2ea84f7a15d790a13a75ede0fecee351ac4a7e/raster/src/main/scala/geotrellis/raster/package.scala#L104-L111)


In [ ]:
tile16.cellType

`NODATA` part of `CellType` effects what values we see:

In [ ]:
tile16.get(0, 0)

In [ ]:
tile16.withNoData(Some(1)).get(0, 0)

In [ ]:
tile16.withNoData(Some(1)).asciiDraw()

Operations on `Tile` use `CellType` to determine "safe" result type.

In [ ]:
IntCellType.union(ShortCellType)

## Tile Interface
Regardless of what type backs the Tile when we access the pixels values they're unified to either `Int` or `Double`. This is done for performance reasons and to facilitate working across cell types. Every method used to access pixel values has `_Double` version (ex: `get` and `getDouble`). The conversion happens on the fly, without affecting the underlying data.

In [ ]:
tile16.get(0,0)
tile16.getDouble(0,0)

In [ ]:
val list = (1 to 16 toArray).toList

def toFloat(x: Int): Float = x.toFloat / 10

list.map(toFloat)

val toFloatValue: Int => Float = { x => x.toFloat / 10 }

In [ ]:
val floats = (1 to 16 toArray).map(x => x.toFloat / 10)
val tileFloat = ArrayTile(floats, cols = 4, rows = 4)
Text(tileFloat.asciiDraw)

In [ ]:
tileFloat.get(0,0)
tileFloat.getDouble(0,0)

Text(tileFloat.asciiDrawDouble(1))

### Tile Interface and NODATA

When using `Tile` methods the `NODATA` cells are converted to either `Int.MinValue` or `Double.NaN` 

**regardless of underlying cell value**

In [ ]:
val tile5ND = tile16.withNoData(Some(5))

In [ ]:
Text(tile5ND.asciiDraw())

In [ ]:
tile5ND.get(0, 1)
tile5ND.getDouble(0, 1)

Use `isData` and `isNoData` macros to test cell values from `Tile`. 

They're macros that are valid for both `Int` and `Double` values.

In [ ]:
var intCount = 0

tile5ND.foreach { v => if (isData(v)) intCount += 1 }

In [ ]:
val tileFND = tileFloat.withNoData(Some(0.5F))

In [ ]:
Text(tileFND.asciiDrawDouble(1))

In [ ]:
var floatCount = 0

tileFND.foreach { v => if (isNoData(v)) floatCount += 1 }

### Actual way to use Tile

In practice you almost never want to use `Tile.get`, there are better ways to do work

In [ ]:
// immutability!
tile5ND.map { v => if(isData(v)) v + 1 else v }.asciiDraw()
// Hey, value at index 5 changed ...

In [ ]:
def f2(left: Int, right: Int): Int = left + right

tile16.combine(tileFloat) { f2 }

In [ ]:
def funct(arg1: Int, arg2: Int): Int = 2


def functCurried(tileFloat: String)(funct: (Double, Double) => Double): Int = 2

funct(2, 3)

val test = functCurried("") _

In [ ]:
tile16.combineDouble(tileFloat) { (a: Double, b: Double) => a + b }

In [ ]:
tile16 * tileFloat // hey, that's safe!

### Mutable Tiles

In [ ]:
val mut = tile16.mutable
mut.set(0, 0, 123)
mut.set(0, 2, 123)

In [ ]:
mut

## Rasters

[`Raster`](https://github.com/locationtech/geotrellis/blob/2f8348ac299d889282b7e6d379eed4696ece1dd7/raster/src/main/scala/geotrellis/raster/Raster.scala#L63) is a composition of `Tile` and `Extent`, placing the tile pixels on a map.

It's up to you to know what projection that raster is in, it does not track `CRS`.

In [ ]:
import geotrellis.vector._

val raster16 = Raster(tile16, Extent(0, 0, 2, 2))

In [ ]:
raster16.cellSize


### RasterExtent

`Raster.rasterExtent` helps you translate between pixel space and map space

In [ ]:
val re = raster16.rasterExtent

In [ ]:
re.gridToMap(0, 0)
re.gridToMap(0, 3)

In [ ]:
re.mapToGrid(0.3, 0.3) // closest cell

You can even rasterizer a geometry:
- http://127.0.0.1:8888/notebooks/work/implicit-methods.ipynb

- [`RasterExtentRasterizeMethods`](https://github.com/locationtech/geotrellis/blob/master/raster/src/main/scala/geotrellis/raster/rasterize/RasterExtentRasterizeMethods.scala)
- available via [implicit method extension](https://github.com/locationtech/geotrellis/blob/1a2ea84f7a15d790a13a75ede0fecee351ac4a7e/raster/src/main/scala/geotrellis/raster/rasterize/Implicits.scala#L26)

In [ ]:
val line = LineString(List(Point(0,0), Point(2, 2)))

re.foreach(line){ (col, row) => println(s"$col, $row") }